In [1]:
%pip install openai --upgrade

  Using cached openai-1.68.2-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.68.2-py3-none-any.whl (606 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import time
from openai import OpenAI

In [2]:
# api 키 불러오기
import os
import openai
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=OPENAI_API_KEY)

In [3]:
assistant_name = "Power Point Generator"
output_file_name = "Presentation001.pptx"
assistant_instruction = r"파일을 만들어줘 {} 언제나. 너는 해당 주제의 전문가이자 PowerPoint 작성 전문가야. 배경, 색상, 글꼴 및 스타일은 현대적이고 읽기 쉽게 1-5개 슬라이드로 만들어줘. 콘텐츠를 매력적으로 만들어줘. 또한 파일을 다운로드할 수 있어야해".format(output_file_name)
prompt_user = "개인 기록을 목표로 등산을 연습하는 사람들을 위한 프레젠테이션을 만들어줘. 유용한 통찰력, 다양한 수준에 대한 교육 계획 및 교육 기간 전, 도중 및 후에 몇 가지 팁이 포함된 프레젠테이션을 만들어줘. 물론 통찰력도 제공해야해"

In [4]:
# OpenAI API에서 비서를 생성하는 함수
assistant = client.beta.assistants.create(
    name=assistant_name,
    instructions=assistant_instruction,
    tools=[{"type": "file_search"},{"type": "code_interpreter"}],
    # 파일 검색과 코드 해석 도구를 사용할 수 있도록 설정
    model="gpt-4-1106-preview"
)

In [5]:
#스레드는 어시스턴트와 사용자 간의 대화 세션
#채팅 창에서 대화를 시작하면 스레드가 생성
#해당 대화 중에 교환되는 모든 메시지는 동일한 스레드에서 진행
thread = client.beta.threads.create()

#메시지는 스레드 내의 통신 단위
#사용자 쿼리, 명령 또는 사용자와 어시스턴트 간에 교환되는 모든 콘텐츠일 수 있습니다.

# 생성된 스레드에 메시지를 추가
message = client.beta.threads.messages.create(
    thread_id=thread.id, #이전에 생성된 스레드의 ID를 참조
    role="user",
    content=prompt_user)

In [6]:
#실행은 어시스턴트와의 단일 상호작용을 의미합니다.
#실행은 사용자가 대화를 시작(스레드 생성)할 때 시작되고 대화가 끝나면 종료됩니다.
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [7]:
timeout = 180       # 최대 대기 시간으로, 180초(3분)로 설정
interval_time = 5   # 실행 상태를 확인하는 주기(5초마다 점검)
time_taken = 0      # 소요시간
while time_taken < timeout:
    run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id)
    # 특정 스레드(thread.id)와 실행(run.id)의 현재 상태를 가져옴

    if run.status == 'completed':
        break
    else:
        print(run.status)
        time.sleep(interval_time)
        time_taken += interval_time

in_progress
in_progress
in_progress
in_progress


In [8]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id)
# thread_id에 해당하는 스레드의 모든 메시지를 가져옴

file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
# 가져온 메시지 중 첫 번째 메시지(messages.data[0])의 첫 번째 콘텐츠(content[0])에서 텍스트(text)에 포함된 주석(annotations)을 확인
# 첫 번째 주석(annotations[0])의 file_path 속성에서 file_id를 추출

file_name = client.files.with_raw_response.retrieve_content(file_path)
# 추출한 file_id를 사용하여 해당 파일의 내용을 서버로부터 가져옴
# retrieve_content는 지정된 파일의 내용을 반환하며, 이때 with_raw_response를 사용하여 원시 응답을 처리

In [9]:
with open(output_file_name, "wb") as file:
    file.write(file_name.content)

In [10]:
# 특정 파일과 어시스턴트를 삭제하는 작업
client.files.delete(file_path)               # file_path로 지정된 파일을 삭제
client.beta.assistants.delete(assistant.id)  # assistant.id로 지정된 어시스턴트를 삭제

AssistantDeleted(id='asst_YFlpUChKoQtU8U6vY7Awskjd', deleted=True, object='assistant.deleted')